In [1]:
import polars as pl
from joblib import Parallel, delayed

# 多源数据并行注入
def ingest_data(source):
    if source.type == "sensor":
        # 读取二进制传感器数据并转换为Parquet
        df = pl.read_ndjson(f"{source.path}/*.bin", schema={"timestamp": pl.Datetime, "value": pl.Float32})
        df.write_parquet(f"lake/sensors/{source.id}.parquet")
        
    elif source.type == "image":
        # 图像元数据存储
        meta = pl.DataFrame({"path": get_image_paths(source)})
        meta.write_parquet(f"lake/images/{source.id}.parquet")

# 并行处理8条生产线的数据
Parallel(n_jobs=8)(delayed(ingest_data)(line) for line in production_lines)

# 数据湖统一查询
def query_lake():
    return (
        pl.scan_parquet("lake/**/*.parquet")
        .filter(pl.col("timestamp") > "2023-01-01")
        .groupby("equipment_id", "hour")
        .agg(pl.col("vibration").mean())
        .collect(streaming=True)
    )

NameError: name 'production_lines' is not defined